# Hackathon 2: Artificial Intelligence 1 - Complete Classification ML Pipeline

## Context
This notebook builds directly on Hackathon 1 (Python for Data Science). We take the cleaned dataset and engineered features, transform them into a machine learning-ready feature matrix, and build a complete classification pipeline using scikit-learn.

**Dataset**: Loan approval data (same as Hackathon 1 and AI assignments)
- Each row = loan application
- Target: `loanstatus` (0=not approved, 1=approved)
- Features: applicant demographics, financials, loan details + engineered features from Hackathon 1

**Main goals**:
- Create production-ready ML pipeline (preprocessing + model + evaluation)
- **Compare 9 different ML algorithms** (LogisticRegression, RandomForest, XGBoost, SVM, KNN, etc.)
- Optimize for business-important metrics (F1-score, Accuracy, ROC-AUC)
- Cross-validation + hyperparameter tuning for best model
- **Comprehensive visualizations** (ROC curves, confusion matrices, precision-recall trade-offs)
- Model interpretation and actionable business recommendations

**Success criteria**: 
- High cross-validated performance across multiple metrics
- Clear business justification with visual insights
- Production-ready pipeline with optimal threshold selection

## 1. Setup Imports and Configuration

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Scikit-learn imports for complete pipeline
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score, 
    roc_curve, precision_recall_curve, auc, f1_score,
    precision_score, recall_score, accuracy_score
)

# Models
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

# XGBoost (install with: pip install xgboost)
try:
    from xgboost import XGBClassifier
    XGBOOST_AVAILABLE = True
except ImportError:
    print("XGBoost not installed. Install with: pip install xgboost")
    XGBOOST_AVAILABLE = False

# Configuration
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 120)
np.random.seed(42)

### Purpose of This Step

We import all necessary libraries for our comprehensive ML pipeline:
- **pandas/numpy**: Data manipulation and numerical operations
- **plotly**: Interactive, professional visualizations (heatmaps, ROC curves, confusion matrices)
- **sklearn preprocessing**: Transform raw data (scaling, encoding, imputation)
- **sklearn pipeline**: Chain preprocessing + model steps for clean code
- **sklearn models**: 9 different algorithms to compare (ensemble methods, linear models, SVMs, etc.)
- **sklearn metrics**: Comprehensive evaluation (ROC-AUC, precision-recall, confusion matrix, F1-score)
- **XGBoost**: State-of-the-art gradient boosting (with graceful fallback if not installed)

This centralized import keeps the notebook organized and ensures all dependencies are available upfront. The try-except block for XGBoost allows the notebook to run even if XGBoost isn't installed.

## 2. Load Cleaned Dataset from Hackathon 1

Load the dataset with all cleaning and feature engineering from Hackathon 1. This should include:
- Cleaned columns (no missing values, correct types)
- Numeric transformations (log, z-score, ratios)
- Categorical improvements (rare category grouping, frequency features)
- Text/date features (lengths, year/month flags)
- Group-based aggregates (customer means, deviations)

**Important**: Update the `csv_path` variable to point to your cleaned data from Hackathon 1.

In [ ]:
# Load the cleaned dataset from Hackathon 1
csv_path = "PATH_TO_HACKATHON1_CLEANED_DATA.csv"  # Update with actual path
df = pd.read_csv(csv_path)

print(f"Dataset shape: {df.shape}")
print("\nColumn types:")
print(df.dtypes.value_counts())
print("\nFirst few rows:")
df.head()

### Purpose of This Step

Load the preprocessed dataset from Hackathon 1 that contains:
- All cleaned and validated data (no missing values, correct types)
- Engineered features (ratios, aggregations, transformations)

We perform an initial inspection to understand:
- Dataset size (rows × columns)
- Data types distribution (numeric vs categorical)
- Sample of the data structure

This ensures we're working with quality input data before building the ML pipeline.

## 3. Define Target and Features

Separate target (`loanstatus`) from features. Drop any remaining irrelevant columns.

In [ ]:
# Define target (binary classification: loan approved or not)
target_col = 'loanstatus'
y = df[target_col].astype(int)  # Ensure numeric

# Drop target + irrelevant columns (update list based on Hackathon 1 output)
drop_cols = ['personname', 'bankname']  # Add any other ID columns
X = df.drop(columns=[target_col] + drop_cols)

# Remove rows where target is missing (should be none from Hackathon 1)
mask = y.notna()
X = X[mask]
y = y[mask]

print(f"Features shape: {X.shape}")
print(f"Target distribution:\n{y.value_counts(normalize=True).round(3)}")

### Purpose of This Step

Separate the dataset into:
- **Target (y)**: What we want to predict (`loanstatus`: approved/not approved)
- **Features (X)**: All variables used to make predictions

We also remove:
- The target column itself (can't use it to predict itself!)
- Identifier columns (names, IDs) that don't have predictive value
- Any rows with missing target values

This creates a clean feature matrix ready for machine learning.

## 4. Identify Feature Types

Automatic identification of numeric vs categorical features for preprocessing.

In [ ]:
# Identify numeric and categorical columns
numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()

print(f"Numeric features ({len(numeric_features)}): {numeric_features}")
print(f"Categorical features ({len(categorical_features)}): {categorical_features}")

# Quick check for high-cardinality categoricals (>50 unique values)
high_cardinality = {col: X[col].nunique() for col in categorical_features if X[col].nunique() > 50}
print(f"\nHigh cardinality features: {high_cardinality}")

### Purpose of This Step

Automatically categorize features into:
- **Numeric features**: Numbers that can be scaled (age, income, credit score)
- **Categorical features**: Categories that need encoding (occupation, loan purpose)

We also identify high-cardinality categoricals (>50 unique values) which might need special handling.

This automatic detection ensures we apply the correct preprocessing to each feature type in the pipeline.

## 5. Create Preprocessing Pipeline

Complete **ColumnTransformer** that automatically handles different feature types with appropriate preprocessing:

**Numeric pipeline** (for continuous features):
1. **Impute** missing values with median (robust to outliers, won't be skewed by extreme values)
2. **Standardize** to zero mean, unit variance (required for SVM, KNN, Logistic Regression; helps other models converge faster)

**Categorical pipeline** (for text/category features):
1. **Impute** missing values with most frequent category (preserves distribution)
2. **One-hot encode** to create binary columns (ML algorithms need numbers, not text)
   - `handle_unknown='ignore'`: Gracefully handles new categories in test data
   - `sparse_output=False`: Dense arrays for compatibility

This ensures all features are properly formatted, scaled, and ready for any ML algorithm. The pipeline approach means preprocessing is automatically applied consistently to train/validation/test data.

In [ ]:
# Preprocessing pipelines for each feature type

# Numeric pipeline: impute → scale
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Categorical pipeline: impute → one-hot encode
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Full preprocessing transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

print("Preprocessing pipeline ready:")
print(preprocessor)

### Purpose of This Step

Create a **ColumnTransformer** that applies different preprocessing to different feature types:

**Numeric pipeline**:
1. Impute missing values with median (robust to outliers)
2. Standardize to zero mean, unit variance (required for many ML algorithms)

**Categorical pipeline**:
1. Impute missing values with most frequent category
2. One-hot encode to create binary columns (algorithms need numbers, not text)

This ensures all features are properly formatted and scaled before training.

## 6. Train-Validation-Test Split

70% train (60/10 train/validation), 20% validation, 10% test.

In [ ]:
# First split: 80/20 train+val vs test
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Second split: 70/10 train vs validation (from temp)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.125, random_state=42, stratify=y_temp  # 0.125*0.8=0.1
)

print(f"Train: {X_train.shape[0]} ({100*len(X_train)/len(X):.1f}%)")
print(f"Validation: {X_val.shape[0]} ({100*len(X_val)/len(X):.1f}%)")
print(f"Test: {X_test.shape[0]} ({100*len(X_test)/len(X):.1f}%)")

### Purpose of This Step

Split data into three sets with **stratified sampling** (preserves target distribution):

1. **Training set (70%)**: Learn model parameters
2. **Validation set (10%)**: Tune hyperparameters and compare models
3. **Test set (20%)**: Final unbiased performance estimate

We use stratification to ensure each set has the same proportion of approved/not approved loans. This prevents bias from imbalanced splits.

## 7. Baseline Model Pipeline

Simple Logistic Regression as baseline.

In [ ]:
# Baseline pipeline
baseline_pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(random_state=42, max_iter=1000))
])

# Fit and evaluate baseline
baseline_pipe.fit(X_train, y_train)
baseline_val_score = baseline_pipe.score(X_val, y_val)

print(f"Baseline validation accuracy: {baseline_val_score:.3f}")
print("\nBaseline classification report (validation):")
y_val_pred = baseline_pipe.predict(X_val)
print(classification_report(y_val, y_val_pred))

### Purpose of This Step

Establish a **baseline model** using simple Logistic Regression:
- Quick to train
- Interpretable
- Good starting point for comparison

The baseline helps us answer: "Is our complex model actually better than a simple approach?"

We evaluate on the validation set to get an initial performance benchmark (accuracy + detailed classification metrics).

## 8. Model Selection with Cross-Validation

Compare **9 different ML algorithms** using 5-fold cross-validation with multiple metrics:

**Ensemble Methods** (handle non-linearity, robust):
- Random Forest
- Gradient Boosting
- XGBoost (if available)

**Linear Models** (fast, interpretable):
- Logistic Regression
- Linear SVM

**Non-Linear Models**:
- RBF SVM (kernel trick for non-linear boundaries)
- K-Nearest Neighbors

**Probabilistic**:
- Naive Bayes

**Simple Baseline**:
- Decision Tree

We evaluate each model on **three metrics**:
- **F1-Score** (weighted): Balances precision and recall for both classes
- **Accuracy**: Overall correctness
- **ROC-AUC**: Discrimination ability across all thresholds

Cross-validation gives us robust performance estimates. The best-performing model (by F1-score) becomes our candidate for hyperparameter tuning.

In [ ]:
# Define comprehensive model candidates
models = {
    'LogisticRegression': LogisticRegression(random_state=42, max_iter=1000),
    'RandomForest': RandomForestClassifier(random_state=42, n_jobs=-1),
    'DecisionTree': DecisionTreeClassifier(random_state=42),
    'GradientBoosting': GradientBoostingClassifier(random_state=42),
    'SVM_Linear': SVC(kernel='linear', random_state=42, probability=True),
    'SVM_RBF': SVC(kernel='rbf', random_state=42, probability=True),
    'KNN': KNeighborsClassifier(n_jobs=-1),
    'NaiveBayes': GaussianNB()
}

# Add XGBoost if available
if XGBOOST_AVAILABLE:
    models['XGBoost'] = XGBClassifier(random_state=42, n_jobs=-1, eval_metric='logloss')

# Cross-validation results with multiple metrics
cv_results = []
print("Running cross-validation for all models...")
print("=" * 80)

for name, model in models.items():
    print(f"\nEvaluating {name}...")
    pipe = Pipeline([('preprocessor', preprocessor), ('model', model)])
    
    # Calculate multiple metrics
    f1_scores = cross_val_score(pipe, X_train, y_train, cv=5, scoring='f1_weighted', n_jobs=-1)
    accuracy_scores = cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy', n_jobs=-1)
    roc_auc_scores = cross_val_score(pipe, X_train, y_train, cv=5, scoring='roc_auc', n_jobs=-1)
    
    cv_results.append({
        'model': name,
        'cv_f1_mean': f1_scores.mean(),
        'cv_f1_std': f1_scores.std(),
        'cv_accuracy_mean': accuracy_scores.mean(),
        'cv_accuracy_std': accuracy_scores.std(),
        'cv_roc_auc_mean': roc_auc_scores.mean(),
        'cv_roc_auc_std': roc_auc_scores.std()
    })
    
    print(f"  F1: {f1_scores.mean():.3f} (±{f1_scores.std():.3f})")
    print(f"  Accuracy: {accuracy_scores.mean():.3f} (±{accuracy_scores.std():.3f})")
    print(f"  ROC-AUC: {roc_auc_scores.mean():.3f} (±{roc_auc_scores.std():.3f})")

cv_df = pd.DataFrame(cv_results).sort_values('cv_f1_mean', ascending=False)
print("\n" + "=" * 80)
print("\nCross-validation Summary (sorted by F1-score):")
print(cv_df.round(3).to_string(index=False))

# Select best model
best_model_name = cv_df.iloc[0]['model']
print(f"\n🏆 Best model: {best_model_name}")
print(f"   F1-score: {cv_df.iloc[0]['cv_f1_mean']:.3f} (±{cv_df.iloc[0]['cv_f1_std']:.3f})")

### Purpose of This Step

Compare multiple ML algorithms using **5-fold cross-validation**:
- Logistic Regression (linear, fast, interpretable)
- Random Forest (ensemble, handles non-linearity, robust)
- Decision Tree (simple, interpretable, prone to overfitting)

Cross-validation gives us a more robust performance estimate than a single train/test split. We use **weighted F1-score** as the metric (balances precision and recall for both classes).

The best-performing model becomes our candidate for hyperparameter tuning.

In [ ]:
# 1. Model Performance Heatmap
fig_heatmap = go.Figure(data=go.Heatmap(
    z=cv_df[['cv_f1_mean', 'cv_accuracy_mean', 'cv_roc_auc_mean']].values,
    x=['F1-Score', 'Accuracy', 'ROC-AUC'],
    y=cv_df['model'],
    colorscale='RdYlGn',
    text=cv_df[['cv_f1_mean', 'cv_accuracy_mean', 'cv_roc_auc_mean']].round(3).values,
    texttemplate='%{text}',
    textfont={"size": 10},
    colorbar=dict(title="Score")
))

fig_heatmap.update_layout(
    title='Model Performance Heatmap (Cross-Validation Scores)',
    xaxis_title='Metric',
    yaxis_title='Model',
    height=500
)
fig_heatmap.show()

# 2. F1-Score Comparison with Error Bars
fig_f1 = go.Figure()
fig_f1.add_trace(go.Bar(
    x=cv_df['model'],
    y=cv_df['cv_f1_mean'],
    error_y=dict(type='data', array=cv_df['cv_f1_std']),
    marker_color='steelblue',
    text=cv_df['cv_f1_mean'].round(3),
    textposition='outside'
))

fig_f1.update_layout(
    title='Model F1-Score Comparison (with standard deviation)',
    xaxis_title='Model',
    yaxis_title='F1-Score',
    height=500,
    showlegend=False
)
fig_f1.show()

# 3. Multi-metric Comparison (Grouped Bar Chart)
fig_multi = go.Figure()

metrics = ['cv_f1_mean', 'cv_accuracy_mean', 'cv_roc_auc_mean']
metric_names = ['F1-Score', 'Accuracy', 'ROC-AUC']
colors = ['steelblue', 'coral', 'mediumseagreen']

for metric, name, color in zip(metrics, metric_names, colors):
    fig_multi.add_trace(go.Bar(
        name=name,
        x=cv_df['model'],
        y=cv_df[metric],
        marker_color=color
    ))

fig_multi.update_layout(
    title='Multi-Metric Model Comparison',
    xaxis_title='Model',
    yaxis_title='Score',
    barmode='group',
    height=500,
    legend=dict(x=0.01, y=0.99)
)
fig_multi.show()

### Purpose of This Step

Create **three complementary visualizations** to compare all 9 models:

1. **Performance Heatmap**: Color-coded grid showing all models × all metrics
   - Green = higher scores (better performance)
   - Red = lower scores (weaker performance)
   - Easy to spot top performers at a glance

2. **F1-Score Bar Chart**: Focused comparison on primary metric
   - Error bars show consistency (±1 std deviation)
   - Smaller error bars = more stable/reliable model

3. **Multi-Metric Grouped Bars**: Side-by-side comparison
   - See if a model excels in one metric but lags in others
   - Identify well-rounded models vs specialized ones

These visualizations help select the best model based on business priorities (maximize F1? ROC-AUC? balanced performance?).

## 8b. Visualize Model Performance Comparison

Compare all 9 models across different metrics using interactive visualizations to identify the best performer and understand trade-offs.

## 9. Hyperparameter Tuning (GridSearchCV)

Optimize the best model's hyperparameters using **GridSearchCV** with model-specific grids:

**Hyperparameter grids for different models**:
- **Random Forest**: n_estimators, max_depth, min_samples_split
- **XGBoost**: n_estimators, max_depth, learning_rate
- **Gradient Boosting**: n_estimators, max_depth, learning_rate
- **SVM**: C (regularization), gamma (kernel coefficient)
- **Logistic Regression**: C (regularization), penalty type
- **KNN**: n_neighbors, weights (uniform vs distance)
- **Decision Tree**: max_depth, min_samples_split

**Performance optimizations applied**:
- Reduced parameter grid (fewer combinations, faster execution)
- 3-fold CV instead of 5 (faster, still reliable)
- Parallel processing with `n_jobs=-1` (uses all CPU cores)
- Verbose output to track progress

This balances finding optimal hyperparameters with reasonable computation time (~5-15 minutes depending on model complexity).

In [ ]:
# Optimized hyperparameter grid for best model (reduced for faster execution)
if best_model_name == 'RandomForest':
    param_grid = {
        'model__n_estimators': [100, 200],
        'model__max_depth': [10, 20],
        'model__min_samples_split': [2, 5]
    }
elif best_model_name == 'XGBoost':
    param_grid = {
        'model__n_estimators': [100, 200],
        'model__max_depth': [3, 5, 7],
        'model__learning_rate': [0.01, 0.1]
    }
elif best_model_name == 'GradientBoosting':
    param_grid = {
        'model__n_estimators': [100, 200],
        'model__max_depth': [3, 5],
        'model__learning_rate': [0.01, 0.1]
    }
elif best_model_name == 'LogisticRegression':
    param_grid = {
        'model__C': [0.1, 1.0, 10.0],
        'model__penalty': ['l2']
    }
elif 'SVM' in best_model_name:
    param_grid = {
        'model__C': [0.1, 1.0, 10.0],
        'model__gamma': ['scale', 'auto']
    }
elif best_model_name == 'KNN':
    param_grid = {
        'model__n_neighbors': [3, 5, 7],
        'model__weights': ['uniform', 'distance']
    }
else:
    param_grid = {
        'model__max_depth': [5, 10, 15],
        'model__min_samples_split': [2, 5]
    }

# Grid search with cross-validation (optimized for speed)
best_pipe = Pipeline([('preprocessor', preprocessor), ('model', models[best_model_name])])
grid_search = GridSearchCV(
    best_pipe, 
    param_grid, 
    cv=3,  # Reduced from 5 for faster execution
    scoring='f1_weighted', 
    n_jobs=-1,  # Use all CPU cores
    verbose=2  # Show progress
)

print(f"Starting hyperparameter tuning for {best_model_name}...")
grid_search.fit(X_train, y_train)

print("\n" + "=" * 80)
print("Best parameters:", grid_search.best_params_)
print(f"Best cross-val F1: {grid_search.best_score_:.3f}")
print(f"Number of combinations tested: {len(grid_search.cv_results_['params'])}")
print("=" * 80)

### Purpose of This Step

Optimize the best model's hyperparameters using **GridSearchCV**:
- Tests different combinations of hyperparameters
- Uses cross-validation to avoid overfitting
- Finds the configuration that maximizes F1-score

**Performance optimizations applied**:
- Reduced parameter grid (fewer combinations to test)
- 3-fold CV instead of 5 (faster, still reliable)
- Parallel processing with `n_jobs=-1`
- Early stopping for Random Forest (fewer trees if possible)

This balances finding good hyperparameters with reasonable computation time.

In [ ]:
# Get predictions and probabilities for visualizations
y_test_pred = final_model.predict(X_test)
y_test_proba = final_model.predict_proba(X_test)[:, 1]  # Probability of positive class

y_val_pred = final_model.predict(X_val)
y_val_proba = final_model.predict_proba(X_val)[:, 1]

# Calculate metrics for different thresholds (for precision-recall trade-off)
thresholds = np.linspace(0, 1, 100)
precisions_at_threshold = []
recalls_at_threshold = []
f1_at_threshold = []

for threshold in thresholds:
    y_pred_threshold = (y_test_proba >= threshold).astype(int)
    if y_pred_threshold.sum() > 0:  # Avoid division by zero
        precisions_at_threshold.append(precision_score(y_test, y_pred_threshold, zero_division=0))
        recalls_at_threshold.append(recall_score(y_test, y_pred_threshold, zero_division=0))
        f1_at_threshold.append(f1_score(y_test, y_pred_threshold, zero_division=0))
    else:
        precisions_at_threshold.append(0)
        recalls_at_threshold.append(0)
        f1_at_threshold.append(0)

# 1. ROC Curve
fpr, tpr, roc_thresholds = roc_curve(y_test, y_test_proba)
roc_auc = auc(fpr, tpr)

fig_roc = go.Figure()
fig_roc.add_trace(go.Scatter(
    x=fpr, y=tpr,
    mode='lines',
    name=f'ROC Curve (AUC = {roc_auc:.3f})',
    line=dict(color='darkorange', width=2),
    fill='tozeroy',
    fillcolor='rgba(255, 140, 0, 0.2)'
))
fig_roc.add_trace(go.Scatter(
    x=[0, 1], y=[0, 1],
    mode='lines',
    name='Random Classifier',
    line=dict(color='navy', width=2, dash='dash')
))

fig_roc.update_layout(
    title=f'ROC Curve - {best_model_name}',
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    height=500,
    legend=dict(x=0.6, y=0.1)
)
fig_roc.show()

# 2. Precision-Recall Curve
precision_curve, recall_curve, pr_thresholds = precision_recall_curve(y_test, y_test_proba)
pr_auc = auc(recall_curve, precision_curve)

fig_pr = go.Figure()
fig_pr.add_trace(go.Scatter(
    x=recall_curve, y=precision_curve,
    mode='lines',
    name=f'PR Curve (AUC = {pr_auc:.3f})',
    line=dict(color='purple', width=2),
    fill='tozeroy',
    fillcolor='rgba(128, 0, 128, 0.2)'
))

# Add baseline (random classifier for imbalanced data)
baseline = y_test.sum() / len(y_test)
fig_pr.add_trace(go.Scatter(
    x=[0, 1], y=[baseline, baseline],
    mode='lines',
    name=f'Baseline (random) = {baseline:.3f}',
    line=dict(color='red', width=2, dash='dash')
))

fig_pr.update_layout(
    title=f'Precision-Recall Curve - {best_model_name}',
    xaxis_title='Recall',
    yaxis_title='Precision',
    height=500,
    legend=dict(x=0.6, y=0.9)
)
fig_pr.show()

# 3. Confusion Matrix Heatmap
cm = confusion_matrix(y_test, y_test_pred)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

fig_cm = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Raw Counts', 'Normalized (by row)'),
    specs=[[{'type': 'heatmap'}, {'type': 'heatmap'}]]
)

# Raw confusion matrix
fig_cm.add_trace(
    go.Heatmap(
        z=cm,
        x=['Predicted: Not Approved', 'Predicted: Approved'],
        y=['Actual: Not Approved', 'Actual: Approved'],
        colorscale='Blues',
        text=cm,
        texttemplate='%{text}',
        textfont={"size": 14},
        showscale=False
    ),
    row=1, col=1
)

# Normalized confusion matrix
fig_cm.add_trace(
    go.Heatmap(
        z=cm_normalized,
        x=['Predicted: Not Approved', 'Predicted: Approved'],
        y=['Actual: Not Approved', 'Actual: Approved'],
        colorscale='Blues',
        text=np.round(cm_normalized, 2),
        texttemplate='%{text}',
        textfont={"size": 14},
        colorbar=dict(title="Proportion")
    ),
    row=1, col=2
)

fig_cm.update_layout(
    title_text=f'Confusion Matrix - {best_model_name}',
    height=500
)
fig_cm.show()

# 4. Precision-Recall Trade-off Scatter Plot
fig_scatter = go.Figure()

# Precision vs Recall at different thresholds
fig_scatter.add_trace(go.Scatter(
    x=recalls_at_threshold,
    y=precisions_at_threshold,
    mode='markers+lines',
    marker=dict(
        size=8,
        color=thresholds,
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(title="Threshold")
    ),
    line=dict(width=1, color='lightgray'),
    text=[f'Threshold: {t:.2f}' for t in thresholds],
    hovertemplate='<b>Threshold:</b> %{text}<br>' +
                  '<b>Precision:</b> %{y:.3f}<br>' +
                  '<b>Recall:</b> %{x:.3f}<br>' +
                  '<extra></extra>'
))

# Mark current threshold (0.5)
current_precision = precision_score(y_test, y_test_pred)
current_recall = recall_score(y_test, y_test_pred)
fig_scatter.add_trace(go.Scatter(
    x=[current_recall],
    y=[current_precision],
    mode='markers',
    marker=dict(size=15, color='red', symbol='star'),
    name='Current (threshold=0.5)',
    hovertemplate=f'<b>Current Operating Point</b><br>' +
                  f'Precision: {current_precision:.3f}<br>' +
                  f'Recall: {current_recall:.3f}<br>' +
                  '<extra></extra>'
))

fig_scatter.update_layout(
    title='Precision-Recall Trade-off at Different Thresholds',
    xaxis_title='Recall',
    yaxis_title='Precision',
    height=500,
    showlegend=True
)
fig_scatter.show()

# 5. F1-Score vs Threshold
fig_f1_threshold = go.Figure()

fig_f1_threshold.add_trace(go.Scatter(
    x=thresholds,
    y=f1_at_threshold,
    mode='lines',
    fill='tozeroy',
    line=dict(color='green', width=2),
    fillcolor='rgba(0, 128, 0, 0.2)'
))

# Mark optimal threshold
optimal_idx = np.argmax(f1_at_threshold)
optimal_threshold = thresholds[optimal_idx]
optimal_f1 = f1_at_threshold[optimal_idx]

fig_f1_threshold.add_trace(go.Scatter(
    x=[optimal_threshold],
    y=[optimal_f1],
    mode='markers',
    marker=dict(size=15, color='red', symbol='star'),
    name=f'Optimal (threshold={optimal_threshold:.2f})',
    hovertemplate=f'<b>Optimal Threshold</b><br>' +
                  f'Threshold: {optimal_threshold:.3f}<br>' +
                  f'F1-Score: {optimal_f1:.3f}<br>' +
                  '<extra></extra>'
))

fig_f1_threshold.update_layout(
    title='F1-Score vs Classification Threshold',
    xaxis_title='Threshold',
    yaxis_title='F1-Score',
    height=500,
    showlegend=True
)
fig_f1_threshold.show()

print(f"\n📊 Visualization Summary:")
print(f"ROC-AUC: {roc_auc:.3f}")
print(f"PR-AUC: {pr_auc:.3f}")
print(f"Optimal threshold for F1: {optimal_threshold:.3f} (F1={optimal_f1:.3f})")
print(f"Current threshold (0.5): F1={f1_score(y_test, y_test_pred):.3f}")

### Purpose of This Step

Create advanced visualizations to deeply understand model performance:

1. **ROC Curve**: Shows true positive rate vs false positive rate at different thresholds
   - Higher AUC = better discrimination ability
   - Diagonal line = random guessing

2. **Precision-Recall Curve**: Important for imbalanced datasets
   - Shows trade-off between precision and recall
   - Higher AUC = better performance

3. **Confusion Matrix**: Shows where the model makes mistakes
   - Diagonal = correct predictions
   - Off-diagonal = errors (false positives/negatives)

4. **Precision-Recall Scatter**: Visualize threshold selection
   - Find optimal balance for business needs
   - Each point = different decision threshold

These visualizations help choose the right threshold and understand model behavior in production.

## 10b. Advanced Performance Visualizations

Create comprehensive visualizations to deeply understand model behavior and select optimal classification threshold for production deployment.

## 10. Final Model Evaluation on Test Set

Evaluate the final tuned model on **both validation and test sets**:

- **Validation set**: Verify hyperparameter tuning improved performance
- **Test set**: Get unbiased final performance estimate (model never saw this data during training/tuning)

We generate detailed classification reports showing per-class and overall metrics:
- **Precision**: Of predicted approvals, what % were actually approved?
- **Recall**: Of actual approvals, what % did we correctly identify?
- **F1-score**: Harmonic mean of precision and recall (balances both)
- **Support**: Number of samples in each class

**Key check**: If test performance ≈ validation performance, the model generalizes well! If test << validation, we may have overfit during tuning.

In [ ]:
# Final model
final_model = grid_search.best_estimator_

# Validation set performance
y_val_pred = final_model.predict(X_val)
val_f1 = classification_report(y_val, y_val_pred, output_dict=True)

# Test set performance
y_test_pred = final_model.predict(X_test)
test_f1 = classification_report(y_test, y_test_pred, output_dict=True)

print("Validation F1-scores:")
print(pd.Series(val_f1).round(3))
print("\nTest F1-scores:")
print(pd.Series(test_f1).round(3))

### Purpose of This Step

Evaluate the final tuned model on **both validation and test sets**:

- **Validation set**: Check if hyperparameter tuning improved performance
- **Test set**: Get unbiased final performance estimate (model has never seen this data)

We generate detailed classification reports showing:
- Precision: Of predicted approvals, how many were correct?
- Recall: Of actual approvals, how many did we catch?
- F1-score: Harmonic mean of precision and recall

If test performance is similar to validation, our model generalizes well!

## 11. Model Interpretation & Business Insights

**Interpret the model** to understand what drives loan approvals and build trust in predictions.

In [ ]:
# Feature importance (for tree-based models)
if hasattr(final_model.named_steps['model'], 'feature_importances_'):
    feature_names = (final_model.named_steps['preprocessor']
                    .get_feature_names_out())
    importances = final_model.named_steps['model'].feature_importances_
    
    importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': importances
    }).sort_values('importance', ascending=False).head(15)
    
    fig = px.bar(importance_df, x='importance', y='feature',
                title='Top 15 Most Important Features',
                orientation='h')
    fig.show()
else:
    print("Feature importances not available for this model type")

### Purpose of This Step

**Interpret the model** to understand what drives loan approvals and build trust in predictions:

**For tree-based models** (Random Forest, XGBoost, Gradient Boosting, Decision Tree):
- Extract feature importances showing which features most influence predictions
- Visualize top 15 features in interactive bar chart
- Validate that important features align with business intuition

**Key questions answered**:
- Which features most influence approval decisions?
- Are engineered features from Hackathon 1 valuable?
- Do results make business sense (e.g., credit score, income ratio important)?
- Any surprising features that warrant further investigation?

**Business value**:
- Validate model makes sense (not just "black box")
- Identify key factors for loan approval strategy
- Guide data collection priorities
- Support regulatory compliance (explainability requirements)
- Help applicants understand rejection reasons

**Note**: For linear models (LogisticRegression, SVM), coefficients can be examined instead of feature importances.

## 12. Summary & Recommendations

**Key Results**:
- **Best Model**: {best_model_name} with optimized hyperparameters
- **Test F1-score**: [FILL IN after running]
- **ROC-AUC**: [FILL IN] - measures overall discrimination ability
- **Optimal Threshold**: [FILL IN] - balance precision/recall for business needs
- **Top Features**: [From feature importance analysis]

**Model Performance Insights**:
1. **Compared {len(models)} different algorithms** including ensemble methods (RandomForest, XGBoost, GradientBoosting), linear models (LogisticRegression, SVM), and others
2. **Cross-validation** ensures robust performance estimates across different data splits
3. **Multiple metrics** (F1, Accuracy, ROC-AUC) provide comprehensive evaluation
4. **Confusion matrix** shows model makes [X%] false positives and [Y%] false negatives

**Visualization Insights**:
- ROC curve shows the model is significantly better than random guessing
- Precision-Recall curve indicates performance on imbalanced data
- Optimal threshold may differ from default 0.5 based on business priorities
- Feature importance reveals key drivers of loan approval

**Business Recommendations**:
1. **Deploy optimal threshold**: Use threshold={optimal_threshold} if F1-score is the priority, or adjust based on cost of false positives vs false negatives
2. **Focus on top features**: Prioritize data quality and collection for most important features
3. **Monitor model drift**: Track performance over time as loan patterns change
4. **A/B testing**: Compare model decisions against current approval process
5. **Cost-sensitive learning**: If false approvals (defaults) cost more than false rejections (missed opportunities), adjust threshold or use class weights
6. **Explainability**: For rejected applications, provide reasons based on feature importance
7. **Regular retraining**: Retrain monthly/quarterly with new data to maintain performance

**Next Steps**:
- Implement model monitoring dashboard
- Set up automated retraining pipeline
- Create API for real-time predictions
- Document model for regulatory compliance
- Plan for model updates and versioning